<img style="float: left; margin: 30px 15px 15px 15px;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFzQj91sOlkeDFkg5HDbjtR4QJYmLXkfMNig&usqp=CAU" width="400" height="500" /> 
    
    
### <font color='navy'> Modelos no lineales para pronósticos. 

**Nombres:** Sergio Daniel Dueñas Godinez y Hector Cesar Ornelas León.

**Fecha:** 15 de Febrero del 2024.

**Expediente** : 739300 y .
    
**Profesor:** Oscar David Jaramillo Zuluaga.
    
**Link Github**: https://github.com/SergioDuenass/Tarea3_SDDUENAS_HCORNELAS

# Tarea 3: Clase 7

## Enunciado de tarea
### Tarea 3


Condensar todos los procedimientos realizados en esta clase donde se implementen los métodos necesarios para crear un correcto flujo de lectura de datos, análisis, ajuste de parámetros, predicción y métricas de errores para un conjunto de datos dado. Algo como lo siguiente:

~~~
class LinearForecast:
    def __init__(self, atributos_del_contructor):
        
    def adf_test(self):
        
    def plot_acf_pac(self):
        
        .
        .
        .
~~~

Implementar los métodos y las lógicas que consideren pertinentes, como de igual forma las posibles variables a introducir en el constructor de la clase.

Posteriormente, utilizar la clase anteriormente creada para generar un modelo SARIMAX para el siguiente conjunto de datos:
https://github.com/quaesito/time-series-forecast-sarimax-prophet/blob/master/A4.xlsx?raw=true

### Ejercicio 1: (enunciado) Transcribirlo

Código de solución estudiante 1

In [4]:
# Código de solución estudiante 1
class LinearForecast:
    def __init__(self, atributos_del_contructor):
        pass
        
    def adf_test(self):
        pass
        
    def plot_acf_pac(self):
        pass
        

Código de solución estudiante 2

In [ ]:
# Código de solución estudiante 2
.
.
.
.


### Respuesta planteada en el ejercicio usando (markdown)
 La respuesta es .........